In [121]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from util import get_data, plot_data

In [244]:
orders_file = "./orders-03.csv"
start_val = 1000000

In [245]:
# read the csv 
orders_df = pd.read_csv(orders_file, index_col='Date', parse_dates=True, na_values=['nan'])

In [246]:
orders_df.sort_index(inplace=True)

In [247]:
end_date = orders_df.index.max()
start_date = orders_df.index.min()
print end_date, start_date

2011-12-20 00:00:00 2011-01-10 00:00:00


In [248]:
stocks = list(orders_df.Symbol.drop_duplicates().values)
adj_close = get_data(stocks, pd.date_range(start_date, end_date))
adj_close = adj_close.fillna(method='ffill')
adj_close = adj_close.fillna(method='bfill')


In [249]:
date_list = adj_close.index
date_list = list(date_list)


In [250]:
comm = 0.
imp = 0.
ret_df = pd.DataFrame(columns=["Portfolio Value"], index=date_list)
emp = np.zeros(len(date_list))
adj_ind = 0
order_ind = 0
order_len = len(orders_df.index)
start_val = 1000000
my_port = Portfolio(start_val, comm, imp, stocks, len(date_list))
while adj_ind < len(date_list):
    
    while order_ind < order_len and orders_df.index[order_ind] == adj_close.index[adj_ind]:
        
        sym = orders_df.Symbol.iloc[order_ind]
        dat = date_list[adj_ind]
        price = adj_close[sym].loc[dat]
        buy = orders_df.Order.iloc[order_ind]
        num = orders_df.Shares.iloc[order_ind]
        print sym, dat, price, buy, num
        if buy == "BUY":
            my_port.buy_stock(sym, price, num, adj_ind+1)
        else:
            my_port.sell_stock(sym, price, num, adj_ind+1)
    
        order_ind += 1

    emp[adj_ind] = my_port.port_value(adj_close.iloc[adj_ind], adj_ind+1)
    adj_ind += 1

    
ret_df["Portfolio Value"] = emp
print ret_df

AAPL 2011-01-10 00:00:00 340.99 BUY 1500
AAPL 2011-01-13 00:00:00 344.2 BUY 1500
GOOG 2011-01-26 00:00:00 616.5 BUY 1000
XOM 2011-02-02 00:00:00 79.96 SELL 4000
XOM 2011-02-10 00:00:00 80.18 BUY 4000
GOOG 2011-03-03 00:00:00 609.56 SELL 1000
IBM 2011-03-03 00:00:00 159.42 SELL 1200
IBM 2011-05-03 00:00:00 168.58 BUY 1500
IBM 2011-06-03 00:00:00 161.67 SELL 3300
AAPL 2011-06-10 00:00:00 324.51 SELL 3000
GOOG 2011-08-01 00:00:00 606.77 BUY 55
GOOG 2011-08-01 00:00:00 606.77 BUY 55
GOOG 2011-12-20 00:00:00 630.37 SELL 110
            Portfolio Value
2011-01-10        1000000.0
2011-01-11         998785.0
2011-01-12        1002940.0
2011-01-13        1004815.0
2011-01-14        1013185.0
2011-01-18         989785.0
2011-01-19         984385.0
2011-01-20         965995.0
2011-01-21         948175.0
2011-01-24         980245.0
2011-01-25         992035.0
2011-01-26         999355.0
2011-01-27         997725.0
2011-01-28         960685.0
2011-01-31         969685.0
2011-02-01         997405.0

In [251]:
class Portfolio:
    
    def __init__(self, cash=0, comm=0, imp=0, stocks=[], lenn=0):
        self.stocks = stocks
        self.commission = comm
        self.impact = imp
        self.arr = np.zeros((lenn+1, len(stocks)+2))
        self.arr[0, -2] = cash
        self.arr[0, -1] = cash
        
   
    
    def buy_stock(self, stock_name, price, number_of_shares, indexx):
        
    
        cash_ind = -2
        port_ind = -1
        
        stock_ind = self.stocks.index(stock_name)
        
        
        self.arr[indexx, stock_ind] = self.arr[0, stock_ind]
        self.arr[indexx, stock_ind] += number_of_shares
        self.arr[0, stock_ind] = self.arr[indexx, stock_ind]
        
        
        self.arr[indexx, cash_ind] = self.arr[0, cash_ind]
        self.arr[indexx, cash_ind] -= price * number_of_shares
        self.arr[0, cash_ind] = self.arr[indexx, cash_ind]
    
        
        self.arr[indexx, cash_ind] -= self.commission
        self.arr[indexx, cash_ind] -= self.impact * number_of_shares * price
        
        
    
    def sell_stock(self, stock_name, price, number_of_shares, indexx):
        # if I already have it
        
        cash_ind = -2
        port_ind = -1
        
        stock_ind = self.stocks.index(stock_name)
        
        
        self.arr[indexx, stock_ind] = self.arr[0, stock_ind]
        self.arr[indexx, stock_ind] -= number_of_shares
        self.arr[0, stock_ind] = self.arr[indexx, stock_ind]
        
        self.arr[indexx, cash_ind] = self.arr[0, cash_ind]
        self.arr[indexx, cash_ind] += price * number_of_shares
        self.arr[0, cash_ind] = self.arr[indexx, cash_ind]
        
        
            
        self.arr[indexx, cash_ind] -= self.commission
        self.arr[indexx, cash_ind] -= self.impact * number_of_shares * price
    
    def port_value(self, adj_close, indexx):

        val = self.arr[0, -2]        
        self.arr[indexx, :] = self.arr[0, :]
        
        for stock in self.stocks:
            price = adj_close[stock]
            stock_ind = self.stocks.index(stock)
            num = self.arr[indexx, stock_ind]
            
            val += num * price
        
        self.arr[indexx, -1] = val
        return val
            
    

In [225]:
my_port.arr[:20]

array([[-1.48000000e+03,  0.00000000e+00,  5.50000000e+01,
         0.00000000e+00,  1.65318145e+06,  1.00000000e+06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+06,  1.00000000e+06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+06,  1.00000000e+06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+06,  1.00000000e+06],
       [-1.50000000e+03,  4.00000000e+03,  0.00000000e+00,
         0.00000000e+00,  9.38100000e+05,  1.00000000e+06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.38100000e+05,  9.38100000e+05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.38100000e+05,  9.38100000e+05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.38100000e+05,  9.38100000e+05],
       [ 0.00000000e+00,  0.00000000e+00,  0.000

In [203]:
my_port

SyntaxError: invalid syntax (<ipython-input-203-2410da17ecf8>, line 1)

In [81]:
a['b'][0]

3

In [83]:
for x in a:
    print a[x]

(3, 200.0)
